In [5]:
import seaborn as sns
import networkx
import folium
import pandas as pd
import osmnx as ox
import ipywidgets as widgets
from folium import plugins
ox.config(use_cache=True, log_console=True)

2022-02-18 09:33:47 Configured OSMnx 1.1.2
2022-02-18 09:33:47 HTTP response caching is on


In [6]:
!jupyter serverextension enable voila
!jupyter server extension enable voila

Enabling: voila
- Writing config: /Users/witoldtenhove/.jupyter
    - Validating...
      voila 0.3.1 OK
Enabling: voila
- Writing config: /Users/witoldtenhove/opt/anaconda3/envs/osmnx/etc/jupyter
    - Validating voila...
      voila 0.3.1 OK
    - Extension successfully enabled.


## Load data

In [7]:
url = 'routes_data.csv'
routes_df = pd.read_csv(url)
routes_df.head()

,route_id,arrival,activity_id,shift_id,longitude,latitude
0,0,1313.393,77.0,6.0,6.164641,52.269232
1,0,1313.393,68.0,5.0,6.164916,52.269976
2,0,1381.537,1.0,0.0,6.164124,52.269695
3,0,1440.034,50.0,3.0,6.172841,52.265654
4,0,2281.557,79.0,6.0,6.172033,52.266394


## Create base map

In [8]:
base_map = folium.Map(location=[routes_df.latitude.mean(
), routes_df.longitude.mean()], zoom_start=15, control_scale=True)
base_map

## Plot all client locations by route (colors)

In [9]:
n_colors = len(pd.unique(routes_df.route_id.values))
colors = sns.color_palette("Set2", n_colors).as_hex()

m = base_map
for index, location_info in routes_df.iterrows():
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=30,
        color="DimGray",
        fill_color=colors[int(location_info["route_id"])],
        fill=True,
        fill_opacity=0.85,
        popup=location_info["route_id"]
    ).add_to(m)
m

## Trace routes on map

In [10]:
deventer_graph = ox.load_graphml("deventer_graph.graphml")

## Function to trace a single route (by route_id) on map

def calc_route(route_id: int, path_coordinates, graph):
    route_id = route_id
    # Get arrays of longitudes and latitudes for path for a given rout_id
    lon_array = path_coordinates[path_coordinates.route_id ==
                                 route_id].longitude
    lat_array = path_coordinates[path_coordinates.route_id ==
                                 route_id].latitude
    gdf_nodes = ox.graph_to_gdfs(graph)[0]

    # Calculates nearest nodes for all path coordinates from arrays
    node_list = ox.distance.nearest_nodes(
        G=deventer_graph,
        X=lon_array,
        Y=lat_array
    )

    # for each pair of subsequent nodes calculate shortest path and append array of path nodes to route array
    route = [[], []]
    for i in range(len(node_list) - 1):
        start = node_list[i]
        end = node_list[i + 1]
        path_nodes = networkx.shortest_path(graph, start, end)
        route[0].append(path_nodes)
        route[1].append(gdf_nodes.loc[path_nodes][['x','y']])
        i += 1

    # return route as list
    return(route)

2022-02-18 09:33:52 Converting node, edge, and graph-level attribute data types
2022-02-18 09:33:52 Loaded graph with 20870 nodes and 39886 edges from "deventer_graph.graphml"


## Plot route on map

In [11]:
route_id = 5

route_list = calc_route(route_id, routes_df, deventer_graph)[0]
route_df_flr = routes_df[routes_df['route_id'] == route_id]
route_map = folium.Map(location=[route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()], control_scale=True)
for index, location_info in route_df_flr.iterrows():
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=20,
        color="DimGray",
        fill_color=colors[int(location_info["route_id"])],
        fill=True,
        fill_opacity=0.85,
        popup=round(location_info["arrival"])
    ).add_to(route_map)

for i, stage in enumerate(route_list):
    route_map = ox.folium.plot_route_folium(
        G=deventer_graph, route=stage, route_map=route_map, color='red', weight=2)

sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
route_map.fit_bounds([sw, ne])
route_map



2022-02-18 09:33:53 Created nodes GeoDataFrame from graph
2022-02-18 09:33:56 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created nodes GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:33:57 Created edges GeoDataFrame from graph
2022-02-18 09:

In [12]:

def trace_route(route_id):
    route_list = calc_route(route_id, routes_df, deventer_graph)[0]
    route_df_flr = routes_df[routes_df['route_id'] == route_id]
    route_map_i = folium.Map(location=[route_df_flr.latitude.mean(
    ), route_df_flr.longitude.mean()], control_scale=True)
    for index, location_info in route_df_flr.iterrows():
        folium.Circle(
            [location_info["latitude"],
             location_info["longitude"]],
            radius=20,
            color="DimGray",
            fill_color=colors[int(location_info["route_id"])],
            fill=True,
            fill_opacity=0.85,
            popup=round(location_info["arrival"])
        ).add_to(route_map_i)

    for i, stage in enumerate(route_list):
        route_map_i = ox.folium.plot_route_folium(
            G=deventer_graph, route=stage, route_map=route_map_i, color='red', weight=2)

    sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
    ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
    route_map_i.fit_bounds([sw, ne])
    display(route_map_i)

In [13]:
route_ids = pd.unique(routes_df.route_id.values)

routeWidget = widgets.RadioButtons(
    options=route_ids,
    description='RouteID:',
    disabled=False
)
    
widgets.interactive(trace_route, route_id=routeWidget)



interactive(children=(RadioButtons(description='RouteID:', options=(0, 1, 2, 3, 4, 5, 6), value=0), Output()),…

## Ant path

In [50]:

# create map

route_id = 5

route_list = calc_route(route_id, routes_df, deventer_graph)[1]
point_list = []
for stage in route_list:
    for point in stage.values:
        point_list.append([point[1], point[0]])
      
route_df_flr = routes_df[routes_df['route_id'] == route_id]
print(route_df_flr)

ant_route_map = folium.Map(location=[route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()], control_scale=True)

for index, location_info in route_df_flr.iterrows():
    fill_color = colors[int(location_info["route_id"])]
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85,
        popup=round(location_info["arrival"])
    ).add_to(ant_route_map)

# Ploting ant-route
ap = plugins.AntPath(point_list, delay='1500', dash_array=['5', '50'], color='Yellow', pulse_color='Red')
ap.add_to(ant_route_map)

start = folium.Marker(
    point_list[0], tooltip=folium.Tooltip('Start', permanent=True)
)
start.add_to(ant_route_map)

finish = folium.Marker(
    point_list[-1], tooltip=folium.Tooltip('Finish', permanent=True)
)
finish.add_to(ant_route_map)

sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
ant_route_map.fit_bounds([sw, ne])
display(ant_route_map)

2022-02-18 10:55:04 Created nodes GeoDataFrame from graph
2022-02-18 10:55:07 Created edges GeoDataFrame from graph
2022-02-18 10:55:07 Created nodes GeoDataFrame from graph
    route_id   arrival  activity_id  shift_id  longitude   latitude
62         5   420.000         58.0       4.0   6.160594  52.267430
63         5   420.000         36.0       2.0   6.161268  52.267151
64         5  1183.406         20.0       1.0   6.165260  52.267300
65         5  2162.731         52.0       3.0   6.166464  52.269238
66         5  2779.272         71.0       5.0   6.166425  52.269844
67         5  3311.956         60.0       4.0   6.169059  52.271503
68         5  3696.765         24.0       1.0   6.168793  52.270316
69         5  3912.071         25.0       1.0   6.167334  52.268616
70         5  4289.686          5.0       0.0   6.166916  52.267690
71         5  4537.609          6.0       0.0   6.171699  52.269982
72         5  5046.272         63.0       4.0   6.170195  52.268510
73        

In [18]:
point_list

[[52.2673744, 6.1607183],
 [52.2670309, 6.1608338],
 [52.2670309, 6.1608338],
 [52.2667557, 6.1610283],
 [52.2672161, 6.1626088],
 [52.2674581, 6.1633258],
 [52.2677442, 6.1640739],
 [52.2667, 6.16521],
 [52.2670099, 6.1660538],
 [52.267482, 6.1655678],
 [52.267524, 6.1654585],
 [52.267524, 6.1654585],
 [52.267665, 6.1658683],
 [52.2671474, 6.1664099],
 [52.2672911, 6.1668008],
 [52.2674713, 6.1672624],
 [52.2675988, 6.1671432],
 [52.2676379, 6.1672594],
 [52.2676823, 6.1673673],
 [52.2677236, 6.1674846],
 [52.2678045, 6.1676908],
 [52.2679168, 6.1679964],
 [52.2682204, 6.16879],
 [52.2684459, 6.1685611],
 [52.2690602, 6.1679288],
 [52.2691553, 6.1678375],
 [52.2695114, 6.1674994],
 [52.2698183, 6.1671569],
 [52.26968, 6.16678],
 [52.2694239, 6.1661545],
 [52.2694239, 6.1661545],
 [52.26968, 6.16678],
 [52.2699887, 6.166489],
 [52.2699887, 6.166489],
 [52.26968, 6.16678],
 [52.2698183, 6.1671569],
 [52.26991, 6.16742],
 [52.27036, 6.16861],
 [52.2707247, 6.1682098],
 [52.2708048, 6.168

# Ipyleaflet

In [87]:
import ipyleaflet as ipl

route_id = 4

route_list = calc_route(route_id, routes_df, deventer_graph)[1]
point_list = []
for stage in route_list:
    for point in stage.values:
        point_list.append([point[1], point[0]])

route_df_flr = routes_df[routes_df['route_id'] == route_id]

m1 = ipl.Map(center=([route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()]))

lines = ipl.Polyline(
    locations=point_list,
    color="tomato",
    fill=False
)

m1.add_layer(lines)

for index, location_info in route_df_flr.iterrows():
    fill_color = colors[int(location_info["route_id"])]
    circle = ipl.Circle(
        location=[location_info["latitude"],location_info["longitude"]],
        name=str(round(location_info["arrival"])),
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85
    )
    m1.add_layer(circle)

start_popup = ipl.Popup(
    location=point_list[0],
    child=widgets.HTML(value='Start'),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m1.add_layer(start_popup)

finish_popup = ipl.Popup(
    location = point_list[-1],
    child=widgets.HTML(value='Finish'),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
m1.add_layer(finish_popup)
control = ipl.LayersControl(position='topright')
m1.add_control(control)
    
msw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
m1.fit_bounds([sw, ne])
display(m1)

m1.save('interactive_route_map.html', title='Route Map')


2022-02-18 11:46:54 Created nodes GeoDataFrame from graph
2022-02-18 11:46:58 Created edges GeoDataFrame from graph
2022-02-18 11:46:58 Created nodes GeoDataFrame from graph


Map(center=[52.26849415073453, 6.166401483360303], controls=(ZoomControl(options=['position', 'zoom_in_text', …